In [2]:
import sympy as sym

In [3]:
# helper functions to be moved later to own module
def create_variables_and_differentials(variables_string, parameter_var_string=None):
    q = sym.symbols(variables_string) # tuple
    if parameter_var_string is None:
        q = sym.Matrix(q)
        dq = sym.Matrix(
            [sym.symbols(f'd{q[i].name}') for i in range(len(q))]
        )
    else:
        if parameter_var_string == '':
            raise ValueError(
                f'{parameter_var_string} -- parameter_var_string, cannot be an empty string, use None!'
            )
        param_var = sym.symbols(parameter_var_string)
        q = sym.Matrix(
            [sym.Function(e.name)(param_var) for e in q]
        )
        dq = sym.diff(q, param_var)
    return q, dq

In [4]:
def line_element_to_metric_tensor(ds_sqrd, dq):
    n = len(dq)
    dq_dq_permutations = sym.tensorproduct(dq, dq).reshape(n ** 2, 1)
    # must expand so coeff method will work properly!
    g = sym.Matrix(
        [sym.expand(ds_sqrd).coeff(e[0], 1) for e in dq_dq_permutations]
    )
    return g.reshape(n, n)


def metric_tensor_to_line_element(g, dq):
    return sym.expand(sym.Mul(sym.Mul(dq.T, g), dq)[0])


### test the two functions are inverses of each other
def test_line_elem_metric_inverses():
    bools = []
    
    # create variables and dq
    c, dt, dx, dy, dz = sym.symbols('c dt dx dy dz')
    dq = sym.Matrix([dt, dx, dy, dz])
    
    # forward direction
    line_element = sym.expand(-c ** 2 * dt ** 2 + dx ** 2 + dy ** 2 + dz ** 2)
    g = line_element_to_metric_tensor(line_element, dq)
    bools.append(
        sym.Equality(line_element, metric_tensor_to_line_element(g, dq))
    )
    # backward direction
    g = sym.Matrix(
        [
            [-c ** 2, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ]
    )
    line_element = metric_tensor_to_line_element(g, dq)
    bools.append(
        sym.Equality(g, line_element_to_metric_tensor(line_element, dq))
    )
    return all(bools)


print(
    f'{test_line_elem_metric_inverses()} -- '
    'line element to metric tensor, and metric tensor to line element are inverse methods.'
)

True -- line element to metric tensor, and metric tensor to line element are inverse methods.


In [5]:
q, dq = create_variables_and_differentials('t, r, theta, phi', parameter_var_string=None)
m = sym.symbols('m')
dq

Matrix([
[    dt],
[    dr],
[dtheta],
[  dphi]])

In [9]:
q

Matrix([
[    t],
[    r],
[theta],
[  phi]])

In [10]:
line_element = (
    (1 - 2 * m / q[1]) * dq[0] ** 2 
    - (1 - 2 * m / q[1]) ** -1 * dq[1] ** 2 
    - q[1] ** 2 * (dq[2] ** 2 + sym.sin(q[2]) ** 2 * dq[3] ** 2)
)
line_element

-dr**2/(-2*m/r + 1) + dt**2*(-2*m/r + 1) - r**2*(dphi**2*sin(theta)**2 + dtheta**2)

In [7]:
g = line_element_to_metric_tensor(line_element, dq)
g

Matrix([
[-2*m/r + 1,               0,     0,                   0],
[         0, -1/(-2*m/r + 1),     0,                   0],
[         0,               0, -r**2,                   0],
[         0,               0,     0, -r**2*sin(theta)**2]])

In [8]:
g[0,0]

-2*m/r + 1